# Step 1: Load and Inspect the Data

In [1]:
import pandas as pd

# Load datasets
production_df = pd.read_csv('cleaned_fao_data_production_indices_data.csv')
land_df = pd.read_csv('cleaned_fao_data_land_data.csv')
fertilizer_df = pd.read_csv('cleaned_fao_data_fertilizers_data.csv')

# Inspect the structure of each dataset
print(production_df.info())
print(land_df.info())
print(fertilizer_df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 234533 entries, 0 to 234532
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   country_or_area  234533 non-null  object 
 1   element_code     234533 non-null  int64  
 2   element          234533 non-null  object 
 3   year             234533 non-null  int64  
 4   unit             234533 non-null  object 
 5   value            234533 non-null  float64
 6   value_footnotes  234533 non-null  object 
 7   category         234533 non-null  object 
dtypes: float64(1), int64(2), object(5)
memory usage: 14.3+ MB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104659 entries, 0 to 104658
Data columns (total 8 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   country_or_area  104659 non-null  object 
 1   element_code     104659 non-null  int64  
 2   element          104659 non-null  object 
 3   ye

# Step 2: Merge Datasets

In [1]:
import pandas as pd

# Load datasets
production_df = pd.read_csv('cleaned_fao_data_production_indices_data.csv')
land_df = pd.read_csv('cleaned_fao_data_land_data.csv')
fertilizer_df = pd.read_csv('cleaned_fao_data_fertilizers_data.csv')

# Select only the important columns
production_columns = ['country_or_area', 'year', 'value']  # Add other necessary columns if needed
land_columns = ['country_or_area', 'year', 'value']  # Adjust based on the important columns for land data
fertilizer_columns = ['country_or_area', 'year', 'value']  # Adjust based on the important columns for fertilizer data

# Rename 'value' columns for clarity before merging
production_df = production_df[production_columns].rename(columns={'value': 'value_prod'})
land_df = land_df[land_columns].rename(columns={'value': 'value_land'})
fertilizer_df = fertilizer_df[fertilizer_columns].rename(columns={'value': 'value_fert'})

# Merge datasets on common columns (country_or_area and year)
merged_df = production_df.merge(land_df, on=['country_or_area', 'year'])
merged_df = merged_df.merge(fertilizer_df, on=['country_or_area', 'year'])

# Inspect the merged dataset
print("Merged Dataset Columns:")
print(merged_df.columns)

print("\nPreview of Merged Dataset:")
print(merged_df.head())


Merged Dataset Columns:
Index(['country_or_area', 'year', 'value_prod', 'value_land', 'value_fert'], dtype='object')

Preview of Merged Dataset:
  country_or_area  year  value_prod  value_land  value_fert
0     Afghanistan  2007   2486910.0     38661.0    920000.0
1     Afghanistan  2007   2486910.0     38661.0    186000.0
2     Afghanistan  2007   2486910.0     38661.0   4357000.0
3     Afghanistan  2007   2486910.0     38661.0      9035.0
4     Afghanistan  2007   2486910.0     38661.0   5387000.0


# Feature Engineering 
Lagged Features: Added lagged variables for value_prod (production indices) to capture temporal trends.
Interaction Features: Combined land use and fertilizer usage as interaction terms.


In [15]:
# Ensure required libraries are imported
import pandas as pd

# Assuming 'merged_df' is the merged dataset containing:
# 'country_or_area', 'year', 'value_prod', 'value_land', 'value_fert'

# 1. Lagged Features: Previous year's productivity
merged_df['value_prod_lag1'] = merged_df.groupby('country_or_area')['value_prod'].shift(1)

# 2. Interaction Term: Combined effect of land and fertilizer
merged_df['land_fert_interaction'] = merged_df['value_land'] * merged_df['value_fert']

# 3. Fertilizer Efficiency: Productivity per unit of fertilizer
merged_df['fertilizer_efficiency'] = merged_df['value_prod'] / merged_df['value_fert']

# 4. Land Productivity: Productivity per unit of land
merged_df['land_productivity'] = merged_df['value_prod'] / merged_df['value_land']

# 5. Year-on-Year Change in Productivity
merged_df['prod_yoy_change'] = merged_df.groupby('country_or_area')['value_prod'].pct_change()

# 6. Normalized Fertilizer Use: Fertilizer use as a proportion of total fertilizer use across all countries
merged_df['normalized_fert'] = merged_df['value_fert'] / merged_df['value_fert'].sum()

# 7. Normalized Land Use: Land use as a proportion of total land use across all countries
merged_df['normalized_land'] = merged_df['value_land'] / merged_df['value_land'].sum()

# Drop rows with NaNs introduced by lagging or percentage changes
merged_df.dropna(inplace=True)

# Preview the enhanced dataset
print("Enhanced Dataset Columns:")
print(merged_df.columns)

print("\nPreview of Enhanced Dataset:")
print(merged_df.head())


Enhanced Dataset Columns:
Index(['country_or_area', 'element_code_x', 'element_x', 'year', 'unit_x',
       'value_prod', 'value_footnotes_x', 'category_x', 'element_code_y',
       'element_y', 'unit_y', 'value_land', 'value_footnotes_y', 'category_y',
       'element_code', 'element', 'unit', 'value_fert', 'value_footnotes',
       'category', 'value_prod_lag1', 'land_fert_interaction',
       'fertilizer_efficiency', 'land_productivity', 'prod_yoy_change',
       'normalized_fert', 'normalized_land'],
      dtype='object')

Preview of Enhanced Dataset:
              country_or_area  element_code_x  \
10340518               Cyprus             438   
16316627           Madagascar             152   
18228962       Western Asia +             438   
8748515   Trinidad and Tobago             154   
7588607             Nicaragua             438   

                                     element_x  year       unit_x  value_prod  \
10340518   Net per capita PIN (base 1999-2001)  1969       Int

## Train-Test split

## Train Predictive Models
Train machine learning models for each goal.

In [16]:
# Check columns in the enhanced dataset
print(merged_df.columns)


Index(['country_or_area', 'element_code_x', 'element_x', 'year', 'unit_x',
       'value_prod', 'value_footnotes_x', 'category_x', 'element_code_y',
       'element_y', 'unit_y', 'value_land', 'value_footnotes_y', 'category_y',
       'element_code', 'element', 'unit', 'value_fert', 'value_footnotes',
       'category', 'value_prod_lag1', 'land_fert_interaction',
       'fertilizer_efficiency', 'land_productivity', 'prod_yoy_change',
       'normalized_fert', 'normalized_land'],
      dtype='object')


In [17]:
from sklearn.model_selection import train_test_split

# Step 1: Define features (X) and target (y)
X = merged_df[['value_land', 'value_fert', 'value_prod_lag1', 'land_fert_interaction', 
               'fertilizer_efficiency', 'land_productivity', 'prod_yoy_change', 
               'normalized_fert', 'normalized_land']]  # Your engineered features
y = merged_df['value_prod']  # Your target variable (productivity)

# Step 2: Train-test split (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Print the shape of the resulting datasets to verify the split
print(f"Training set features: {X_train.shape}")
print(f"Testing set features: {X_test.shape}")
print(f"Training set target: {y_train.shape}")
print(f"Testing set target: {y_test.shape}")


Training set features: (943446, 9)
Testing set features: (235862, 9)
Training set target: (943446,)
Testing set target: (235862,)


## Goal 1: Optimize Farm Input Usage
Use a regression model to predict productivity based on fertilizer and land use.

### 1. Check for NaN or infinite values

In [19]:
# Check for NaN or infinite values in the training set
print("Checking for NaNs in the training features:")
print(X_train.isna().sum())  # To check for NaNs in features
print(np.isinf(X_train).sum())  # To check for infinite values in features

# Check target variable for NaN or infinite values
print("Checking for NaNs in the target:")
print(y_train.isna().sum())  # To check for NaNs in target
print(np.isinf(y_train).sum())  # To check for infinite values in target


Checking for NaNs in the training features:
value_land               0
value_fert               0
value_prod_lag1          0
land_fert_interaction    0
fertilizer_efficiency    0
land_productivity        0
prod_yoy_change          0
normalized_fert          0
normalized_land          0
dtype: int64
value_land                   0
value_fert                   0
value_prod_lag1              0
land_fert_interaction        0
fertilizer_efficiency    33333
land_productivity          393
prod_yoy_change            919
normalized_fert              0
normalized_land              0
dtype: int64
Checking for NaNs in the target:
0
0


### 2. Fill NaNs with the median value of each column

In [20]:
# Fill NaNs with the median value of each column
X_train.fillna(X_train.median(), inplace=True)


###  3. Clip the values to a reasonable range, e.g., between -1e6 and 1e6

In [21]:
# Clip the values to a reasonable range, e.g., between -1e6 and 1e6
X_train = X_train.clip(-1e6, 1e6)
X_test = X_test.clip(-1e6, 1e6)


### 4. Check for Large Numerical Values

In [22]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# Fit and transform the training data
X_train_scaled = scaler.fit_transform(X_train)

# Transform the test data using the same scaler
X_test_scaled = scaler.transform(X_test)


### 5. Train the model for Goal 1

In [23]:
# Train the model
regressor = LinearRegression()
regressor.fit(X_train_scaled, y_train)

# Predict on the test set
y_pred = regressor.predict(X_test_scaled)

# Evaluate the model
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# Print the evaluation metrics
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"R-squared (R²): {r2:.2f}")


Mean Absolute Error (MAE): 4503872.13
R-squared (R²): 0.55


## Goal 2: Identify Sustainable Land Use Practices
Classify land use practices as "sustainable" or "unsustainable."

In [2]:
import pandas as pd

# Load datasets
production_df = pd.read_csv('cleaned_fao_data_production_indices_data.csv')
land_df = pd.read_csv('cleaned_fao_data_land_data.csv')
fertilizer_df = pd.read_csv('cleaned_fao_data_fertilizers_data.csv')

# Select only the important columns
production_columns = ['country_or_area', 'year', 'value']  # Add other necessary columns if needed
land_columns = ['country_or_area', 'year', 'value']  # Adjust based on the important columns for land data
fertilizer_columns = ['country_or_area', 'year', 'value']  # Adjust based on the important columns for fertilizer data

# Rename 'value' columns for clarity before merging
production_df = production_df[production_columns].rename(columns={'value': 'value_prod'})
land_df = land_df[land_columns].rename(columns={'value': 'value_land'})
fertilizer_df = fertilizer_df[fertilizer_columns].rename(columns={'value': 'value_fert'})

# Merge datasets on common columns (country_or_area and year)
merged_df = production_df.merge(land_df, on=['country_or_area', 'year'])
merged_df = merged_df.merge(fertilizer_df, on=['country_or_area', 'year'])

# Create a binary target
merged_df['sustainable'] = (merged_df['value_prod'] > merged_df['value_prod'].median()).astype(int)

# Train-test split for classification
X = merged_df[features]
y = merged_df['sustainable']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

# Evaluate the model
y_pred = rf_classifier.predict(X_test)
print(classification_report(y_test, y_pred))


NameError: name 'features' is not defined

## Goal 3: Foprecast Sustainable Agriculture Productivity
Build a time-series forecasting model.

In [3]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

# Focus on a specific country for time-series modeling
country_data = merged_df[merged_df['country_or_area'] == 'India']

# Train a SARIMAX model
sarimax_model = SARIMAX(country_data['value_prod'], 
                        exog=country_data[['value_land', 'value_fert']], 
                        order=(1, 1, 1), 
                        seasonal_order=(1, 1, 1, 12))
sarimax_results = sarimax_model.fit()

# Forecast future productivity
forecast = sarimax_results.forecast(steps=12, exog=country_data[['value_land', 'value_fert']].iloc[-12:])
print(forecast)

ModuleNotFoundError: No module named 'statsmodels'

# Step 7:Visualize model predictions and feature importance for insights.

In [4]:
import matplotlib.pyplot as plt

# Feature importance from Random Forest
importances = rf_model.feature_importances_
plt.barh(features, importances)
plt.title("Feature Importance")
plt.show()

# Compare actual vs predicted productivity
plt.plot(y_test.values, label="Actual")
plt.plot(y_pred, label="Predicted")
plt.legend()
plt.title("Actual vs Predicted Productivity")
plt.show()

AttributeError: 'RandomForestRegressor' object has no attribute 'estimators_'